In [2]:
import json, math
from collections import defaultdict
import numpy as np
import pandas as pd
import community as community_louvain
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
INPUT_PATH = "MAPQGE30/chr1_1mb.RAWobserved"  
K = 10  #change K to get different number of clusters                               

In [13]:
data = np.loadtxt(INPUT_PATH)
if data.shape[1] < 3:
    raise ValueError("Expected 3 columns (i, j, v).")
i = data[:, 0].astype(int)
j = data[:, 1].astype(int)
w = data[:, 2].astype(float)

In [5]:
nbrs = defaultdict(list)
for a, b, wt in zip(i, j, w):
    if a == b:
        continue  # drop self edges
    nbrs[a].append((b, wt)) #add to adjacency list
    nbrs[b].append((a, wt)) #add to adjacency list
nbrs

defaultdict(list,
            {np.int64(0): [(np.int64(1000000), np.float64(46354.0)),
              (np.int64(2000000), np.float64(4065.0)),
              (np.int64(3000000), np.float64(2443.0)),
              (np.int64(4000000), np.float64(888.0)),
              (np.int64(5000000), np.float64(783.0)),
              (np.int64(6000000), np.float64(2319.0)),
              (np.int64(7000000), np.float64(905.0)),
              (np.int64(8000000), np.float64(1207.0)),
              (np.int64(9000000), np.float64(1445.0)),
              (np.int64(10000000), np.float64(1023.0)),
              (np.int64(11000000), np.float64(1176.0)),
              (np.int64(12000000), np.float64(839.0)),
              (np.int64(13000000), np.float64(102.0)),
              (np.int64(14000000), np.float64(490.0)),
              (np.int64(15000000), np.float64(755.0)),
              (np.int64(16000000), np.float64(1071.0)),
              (np.int64(17000000), np.float64(661.0)),
              (np.int64(18000000)

In [14]:
topk = {} #each node's top K neighbors
for node, lst in nbrs.items():
    lst_sorted = sorted(lst, key=lambda x: (-x[1], x[0]))  # weight desc, tiebreak by id
    topk[node] = lst_sorted[:K]
topk

{np.int64(0): [(np.int64(1000000), np.float64(46354.0)),
  (np.int64(2000000), np.float64(4065.0)),
  (np.int64(3000000), np.float64(2443.0)),
  (np.int64(6000000), np.float64(2319.0)),
  (np.int64(9000000), np.float64(1445.0)),
  (np.int64(8000000), np.float64(1207.0)),
  (np.int64(11000000), np.float64(1176.0)),
  (np.int64(16000000), np.float64(1071.0)),
  (np.int64(10000000), np.float64(1023.0)),
  (np.int64(7000000), np.float64(905.0))],
 np.int64(1000000): [(np.int64(2000000), np.float64(59246.0)),
  (np.int64(0), np.float64(46354.0)),
  (np.int64(3000000), np.float64(24534.0)),
  (np.int64(6000000), np.float64(14231.0)),
  (np.int64(4000000), np.float64(8705.0)),
  (np.int64(5000000), np.float64(7691.0)),
  (np.int64(9000000), np.float64(6871.0)),
  (np.int64(11000000), np.float64(6074.0)),
  (np.int64(7000000), np.float64(6059.0)),
  (np.int64(8000000), np.float64(5861.0))],
 np.int64(2000000): [(np.int64(3000000), np.float64(118157.0)),
  (np.int64(1000000), np.float64(59246.0

In [15]:
edges = set() #only unique edges
for a, lst in topk.items():
    for b, wt in lst:
        u, v = (a, b) if a < b else (b, a)
        edges.add((u, v))
edges

{(np.int64(59000000), np.int64(67000000)),
 (np.int64(208000000), np.int64(212000000)),
 (np.int64(64000000), np.int64(65000000)),
 (np.int64(217000000), np.int64(218000000)),
 (np.int64(148000000), np.int64(149000000)),
 (np.int64(223000000), np.int64(227000000)),
 (np.int64(232000000), np.int64(233000000)),
 (np.int64(163000000), np.int64(164000000)),
 (np.int64(189000000), np.int64(192000000)),
 (np.int64(247000000), np.int64(248000000)),
 (np.int64(50000000), np.int64(56000000)),
 (np.int64(204000000), np.int64(207000000)),
 (np.int64(7000000), np.int64(15000000)),
 (np.int64(65000000), np.int64(71000000)),
 (np.int64(239000000), np.int64(241000000)),
 (np.int64(235000000), np.int64(246000000)),
 (np.int64(170000000), np.int64(172000000)),
 (np.int64(149000000), np.int64(155000000)),
 (np.int64(31000000), np.int64(36000000)),
 (np.int64(48000000), np.int64(58000000)),
 (np.int64(185000000), np.int64(187000000)),
 (np.int64(201000000), np.int64(211000000)),
 (np.int64(57000000), np.

In [16]:
nodes = sorted({x for e in edges for x in e})


In [17]:
G = nx.Graph()
G.add_edges_from([(u, v) for (u, v) in edges])  # edges from your top-k code
G

In [18]:
partition = community_louvain.best_partition(G)
partition

{np.int64(59000000): 0,
 np.int64(67000000): 8,
 np.int64(208000000): 2,
 np.int64(212000000): 2,
 np.int64(64000000): 0,
 np.int64(65000000): 8,
 np.int64(217000000): 2,
 np.int64(218000000): 2,
 np.int64(148000000): 4,
 np.int64(149000000): 4,
 np.int64(223000000): 2,
 np.int64(227000000): 5,
 np.int64(232000000): 5,
 np.int64(233000000): 5,
 np.int64(163000000): 6,
 np.int64(164000000): 6,
 np.int64(189000000): 7,
 np.int64(192000000): 7,
 np.int64(247000000): 5,
 np.int64(248000000): 5,
 np.int64(50000000): 0,
 np.int64(56000000): 0,
 np.int64(204000000): 2,
 np.int64(207000000): 2,
 np.int64(7000000): 1,
 np.int64(15000000): 1,
 np.int64(71000000): 8,
 np.int64(239000000): 5,
 np.int64(241000000): 5,
 np.int64(235000000): 5,
 np.int64(246000000): 5,
 np.int64(170000000): 6,
 np.int64(172000000): 6,
 np.int64(155000000): 4,
 np.int64(31000000): 1,
 np.int64(36000000): 0,
 np.int64(48000000): 0,
 np.int64(58000000): 0,
 np.int64(185000000): 6,
 np.int64(187000000): 7,
 np.int64(2010

In [19]:
d3 = {
    "nodes": [{"id": int(n), "community": int(partition.get(n, -1))} for n in nodes],
    "links": [{"source": int(u), "target": int(v)} for (u, v) in sorted(edges)],
}

json_out = f"graph_knn_k{K}.json"
with open(json_out, "w") as f:
    json.dump(d3, f)

In [25]:
len(edges)

1340